### Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)
Explore a geographical location, over the course of two weeks, you will have the opportunity to be as creative as you want and come up with an idea to leverage the Foursquare location data to explore or compare neighborhoods or cities of your choice or to come up with a problem that you can use the Foursquare location data to solve. If you cannot think of an idea or a problem, here are some ideas to get you started:

In Module 3, we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. Is New York City more like Toronto or Paris or some other multicultural city? I will leave it to you to refine this idea.
In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?

These are just a couple of many ideas and problems that can be solved using location data in addition to other datasets. No matter what you decide to do, make sure to provide sufficient justification of why you think what you want to do or solve is important and why would a client or a group of people be interested in your project.

In [1]:


from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import folium

from bs4 import BeautifulSoup
 
    
from sklearn.cluster import KMeans

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

from sklearn.datasets.samples_generator import make_blobs
import matplotlib.cm as cm
import matplotlib.colors as colors

C:\Users\gregoryt\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# Greece towns
url='https://www.latlong.net/category/cities-86-15.html'
GR_df=pd.read_html(url, header=0)[0]
GR_df.head(10)

,Place Name,Latitude,Longitude
0,"Volos, Magnesia, Greece",39.366669,22.933332
1,"Kalamata, Messenia, Greece",37.036598,22.114401
2,"Komotini, East Macedonia and Thrace, Greece",41.122440,25.406557
3,"Kallithea, Attica, Greece",37.955894,23.702099
4,"Corinth, Corinthia, Greece",37.940880,22.944860
5,"Athens, Greece",37.983810,23.727539
6,"Glyfada, Attica, Greece",37.865044,23.755045
7,"Evosmos, Greece",40.666138,22.903774
8,"Thessaloniki, Greece",40.629269,22.947412
9,"Lamia, Greece",38.895973,22.434900


In [3]:
df = GR_df.rename(columns = {"Place Name": "Neighborhood" })
 
df

,Neighborhood,Latitude,Longitude
0,"Volos, Magnesia, Greece",39.366669,22.933332
1,"Kalamata, Messenia, Greece",37.036598,22.114401
2,"Komotini, East Macedonia and Thrace, Greece",41.122440,25.406557
3,"Kallithea, Attica, Greece",37.955894,23.702099
4,"Corinth, Corinthia, Greece",37.940880,22.944860
5,"Athens, Greece",37.983810,23.727539
6,"Glyfada, Attica, Greece",37.865044,23.755045
7,"Evosmos, Greece",40.666138,22.903774
8,"Thessaloniki, Greece",40.629269,22.947412
9,"Lamia, Greece",38.895973,22.434900


In [4]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Greece'

geolocator = Nominatim(user_agent="Greece_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinates of Greece are 38.9953683, 21.9877132.


In [5]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### foursquare.com

In [6]:
CLIENT_ID = 'QU32LQZYEDZCUMQPH03ZX2OWCSZU4IYX11S0YS5RP0W2GJIM' # your Foursquare ID
CLIENT_SECRET = 'B4MZI3QQZIQD4QGCV4QPNUY0CYLH4VEVSZRIGZBFYPZU4GMR' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
 

In [7]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
GR_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Volos, Magnesia, Greece
Kalamata, Messenia, Greece
Komotini, East Macedonia and Thrace, Greece
Kallithea, Attica, Greece
Corinth, Corinthia, Greece
Athens, Greece
Glyfada, Attica, Greece
Evosmos, Greece
Thessaloniki, Greece
Lamia, Greece
Karditsa, Gerece
Arta, Greece
Ioannina, Greece
Patra, Greece
Alexandroupolis, Greece
Argos, Greece
Lamia, Greece
Alexandroupolis, Greece
Larissa, Greece
Heraklion, Crete, Greece
Ermioni, Greece


In [9]:
GR_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Volos, Magnesia, Greece",39.366669,22.933332,Αββαείο,39.364999,22.932674,Beer Bar
1,"Volos, Magnesia, Greece",39.366669,22.933332,Gym Way Cross Fitness Club,39.366767,22.930734,Gym / Fitness Center
2,"Volos, Magnesia, Greece",39.366669,22.933332,Village 4 Cinemas,39.365845,22.934775,Multiplex
3,"Volos, Magnesia, Greece",39.366669,22.933332,Μουσειον Εκλεκτοποιειον,39.364219,22.935708,Meze Restaurant
4,"Volos, Magnesia, Greece",39.366669,22.933332,Αίθριον - Ethrion Day&Night,39.364260,22.931541,Modern Greek Restaurant
...,...,...,...,...,...,...,...
891,"Ermioni, Greece",37.384892,23.246469,Cavo Bianco,37.382683,23.248905,Bar
892,"Ermioni, Greece",37.384892,23.246469,Γανώσης,37.385308,23.249440,Greek Restaurant
893,"Ermioni, Greece",37.384892,23.246469,Veggera Restaurant,37.384080,23.247061,Pizza Place
894,"Ermioni, Greece",37.384892,23.246469,Αίγλη,37.383749,23.247444,Coffee Shop


In [10]:
### or
GR_venues.shape

(896, 7)

In [11]:
GR_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alexandroupolis, Greece",139,139,139,139,139,139
"Argos, Greece",1,1,1,1,1,1
"Arta, Greece",43,43,43,43,43,43
"Athens, Greece",100,100,100,100,100,100
"Corinth, Corinthia, Greece",6,6,6,6,6,6
"Ermioni, Greece",21,21,21,21,21,21
"Evosmos, Greece",13,13,13,13,13,13
"Glyfada, Attica, Greece",65,65,65,65,65,65
"Heraklion, Crete, Greece",20,20,20,20,20,20


In [12]:
GR_CV = pd.get_dummies(GR_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
GR_CV['Neighborhood'] = GR_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [GR_CV.columns[-1]] + list(GR_CV.columns[:-1])
GR_CV = GR_CV[fixed_columns]

GR_CV.head()

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bar,Basketball Court,...,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Train Station,Tsipouro Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar
0,"Volos, Magnesia, Greece",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Volos, Magnesia, Greece",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Volos, Magnesia, Greece",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Volos, Magnesia, Greece",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Volos, Magnesia, Greece",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
GR_CV_grouped = GR_CV.groupby('Neighborhood').mean().reset_index()
GR_CV_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bar,Basketball Court,...,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Train Station,Tsipouro Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar
0,"Alexandroupolis, Greece",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.021583,0.021583,0.000000,...,0.007194,0.000000,0.00,0.000000,0.014388,0.000000,0.000000,0.000000,0.000000,0.014388
1,"Argos, Greece",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Arta, Greece",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.023256,0.023256,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000
3,"Athens, Greece",0.000000,0.01,0.000000,0.01,0.000000,0.000000,0.030000,0.020000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000
4,"Corinth, Corinthia, Greece",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Ermioni, Greece",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.095238,0.047619,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Evosmos, Greece",0.000000,0.00,0.000000,0.00,0.076923,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Glyfada, Attica, Greece",0.015385,0.00,0.000000,0.00,0.000000,0.000000,0.061538,0.015385,0.000000,...,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.000000,0.015385,0.000000,0.030769
8,"Heraklion, Crete, Greece",0.000000,0.00,0.000000,0.00,0.050000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Ioannina, Greece",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.070000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000


In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = GR_CV_grouped['Neighborhood']

for ind in np.arange(GR_CV_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(GR_CV_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alexandroupolis, Greece",Café,Greek Restaurant,Coffee Shop,Souvlaki Shop,Hotel,Clothing Store,Nightclub,Seafood Restaurant,Dessert Shop,Bakery
1,"Argos, Greece",Bookstore,Wine Bar,Flower Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food & Drink Shop
2,"Arta, Greece",Café,Meze Restaurant,Ouzeri,Plaza,Greek Restaurant,Pizza Place,Clothing Store,Church,Comfort Food Restaurant,Frozen Yogurt Shop
3,"Athens, Greece",Hotel,Coffee Shop,Theater,Greek Restaurant,Café,Falafel Restaurant,Bakery,Bookstore,Dessert Shop,Plaza
4,"Corinth, Corinthia, Greece",Gym,Bakery,Sports Club,Supermarket,Shop & Service,Soccer Field,Escape Room,Electronics Store,Fruit & Vegetable Store,Frozen Yogurt Shop


In [16]:
# set number of clusters
kclusters = 7

GR_grouped_clustering = GR_CV_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(GR_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 2, 4, 6, 5, 4, 1, 6, 3, 4])

In [23]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head(1)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,"Alexandroupolis, Greece",Café,Greek Restaurant,Coffee Shop,Souvlaki Shop,Hotel,Clothing Store,Nightclub,Seafood Restaurant,Dessert Shop,Bakery


In [24]:
# add clustering labels
GR_merged = GR_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
all_merged = GR_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

all_merged 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Volos, Magnesia, Greece",39.366669,22.933332,Αββαείο,39.364999,22.932674,Beer Bar,6,Music Venue,Coffee Shop,Gastropub,Beer Bar,Nightclub,Plaza,Creperie,Multiplex,Modern Greek Restaurant,Meze Restaurant
1,"Volos, Magnesia, Greece",39.366669,22.933332,Gym Way Cross Fitness Club,39.366767,22.930734,Gym / Fitness Center,6,Music Venue,Coffee Shop,Gastropub,Beer Bar,Nightclub,Plaza,Creperie,Multiplex,Modern Greek Restaurant,Meze Restaurant
2,"Volos, Magnesia, Greece",39.366669,22.933332,Village 4 Cinemas,39.365845,22.934775,Multiplex,6,Music Venue,Coffee Shop,Gastropub,Beer Bar,Nightclub,Plaza,Creperie,Multiplex,Modern Greek Restaurant,Meze Restaurant
3,"Volos, Magnesia, Greece",39.366669,22.933332,Μουσειον Εκλεκτοποιειον,39.364219,22.935708,Meze Restaurant,6,Music Venue,Coffee Shop,Gastropub,Beer Bar,Nightclub,Plaza,Creperie,Multiplex,Modern Greek Restaurant,Meze Restaurant
4,"Volos, Magnesia, Greece",39.366669,22.933332,Αίθριον - Ethrion Day&Night,39.364260,22.931541,Modern Greek Restaurant,6,Music Venue,Coffee Shop,Gastropub,Beer Bar,Nightclub,Plaza,Creperie,Multiplex,Modern Greek Restaurant,Meze Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,"Ermioni, Greece",37.384892,23.246469,Cavo Bianco,37.382683,23.248905,Bar,4,Café,Greek Restaurant,Pizza Place,Bakery,Taverna,Bar,Port,Supermarket,Fish Taverna,Coffee Shop
892,"Ermioni, Greece",37.384892,23.246469,Γανώσης,37.385308,23.249440,Greek Restaurant,4,Café,Greek Restaurant,Pizza Place,Bakery,Taverna,Bar,Port,Supermarket,Fish Taverna,Coffee Shop
893,"Ermioni, Greece",37.384892,23.246469,Veggera Restaurant,37.384080,23.247061,Pizza Place,4,Café,Greek Restaurant,Pizza Place,Bakery,Taverna,Bar,Port,Supermarket,Fish Taverna,Coffee Shop
894,"Ermioni, Greece",37.384892,23.246469,Αίγλη,37.383749,23.247444,Coffee Shop,4,Café,Greek Restaurant,Pizza Place,Bakery,Taverna,Bar,Port,Supermarket,Fish Taverna,Coffee Shop


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster , venue in zip(all_merged['Neighborhood Latitude'], all_merged['Neighborhood Longitude'], all_merged['Neighborhood'], all_merged['Cluster Labels'], all_merged['Venue']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + ' Venue ' + str(venue), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters